# Using FastText via Gensim
This notebook is modified based on the original one provided by Gensim.  
You can find it here: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/FastText_Tutorial.ipynb

This tutorial is about using [fastText](https://github.com/facebookresearch/fastText) model in Gensim. There are two ways you can use fastText in Gensim:  
  
**1. Gensim's native implementation of fastText**  
**2. Gensim wrapper for fastText's original C++ code**  
  
Here, we'll learn to work with fastText library for training word-embedding models, saving & loading them and performing similarity operations & vector lookups analogous to Word2Vec.

## When to use FastText?
The main principle behind fastText is that the **morphological structure** of a word carries important information about the meaning of the word, which is not taken into account by traditional word embeddings, which train a unique word embedding for every individual word.  
This is especially significant for **morphologically rich languages (German, Turkish)** in which a single word can have a large number of morphological forms, each of which might occur rarely, thus making it hard to train good word embeddings.  
fastText attempts to solve this by **treating each word as the aggregation of its subwords**. For the sake of simplicity and language-independence, subwords are taken to be the character ngrams of the word. **The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.**  
According to a detailed comparison of Word2Vec and FastText in [this notebook](Word2Vec_FastText_Comparison.ipynb), fastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially **when the size of the training corpus is small**. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.  
Training time for fastText is significantly higher than the Gensim version of Word2Vec (`15min 42s` vs `6min 42s` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).  
fastText can be used to obtain vectors for **out-of-vocabulary (OOV) words**, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.

## Training models

For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim) for training our model.  
For using the wrapper for fastText, you need to have fastText setup locally to be able to train models. See [installation instructions for fastText](https://github.com/facebookresearch/fastText/#requirements) if you don't have fastText installed already.

## Using Gensim's implementation of fastText

In [1]:
import gensim
import os
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText as FT_gensim

Set file names for train and test data

In [2]:
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = data_dir + 'lee_background.cor'
lee_data = LineSentence(lee_train_file)

### Quick test

In [3]:
from gensim.models import FastText

sentences_1 = [["cat", "say", "meow"], ["dog", "say", "woof"]]
sentences_2 = [["dude", "say", "wazzup!"]]

model = FastText(min_count=1)
model.build_vocab(sentences_1)
model.train(sentences_1, total_examples=model.corpus_count, epochs=model.iter)
model.build_vocab(sentences_2, update=True)
model.train(sentences_2, total_examples=model.corpus_count, epochs=model.iter)

### check the ngrams

In [13]:
import numpy as np

In [14]:
np.array(list(model.wv.ngrams))

array(['og>', '<du', 'dud', '<wa', 'say', 'dude>', '<say>', 'azzup',
       '<dud', '<cat', '<meow', 'eow>', '<say', 'zup!', 'de>', '<ca',
       '<do', '<wazzu', 'dude', '<me', 'p!>', 'up!', 'woof', 'woo', 'ay>',
       '<woof', 'waz', 'at>', '<wazz', 'of>', 'zzu', 'up!>', 'zup!>',
       'wazzup', 'oof', 'azzup!', 'ude>', '<dude', 'say>', 'meo', 'zup',
       'zzup', 'ow>', '<dude>', '<dog>', 'ude', 'zzup!>', '<woo', 'wazz',
       'zzup!', '<meo', 'azzu', 'woof>', '<wo', 'meow', '<woof>', 'dog',
       'cat', 'dog>', 'azz', '<cat>', 'eow', '<dog', '<sa', 'wazzu',
       '<waz', 'oof>', '<meow>', 'cat>', 'meow>'],
      dtype='|S6')

### train on lee dataset

In [17]:
model_gensim = FastText(size=100)

# build the vocabulary
model_gensim.build_vocab(lee_data)

# train the model
model_gensim.train(lee_data, total_examples=model_gensim.corpus_count, epochs=model_gensim.iter)

print(model_gensim)

FastText(vocab=1762, size=100, alpha=0.025)


In [20]:
np.array(list(model_gensim.wv.ngrams))[:10]

array([u'<stru', u'<cal', u'<cam', u'<can', u'hree>', u'<Shan', u'<cab',
       u'Pollo', u'foun', u'<brou'],
      dtype='<U6')

## Training hyperparameters

Hyperparameters for training the model follow **the same pattern as Word2Vec**. FastText supports the folllowing parameters from the original word2vec - 
     - model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
     - size: Size of embeddings to be learnt (Default 100)
     - alpha: Initial learning rate (Default 0.025)
     - window: Context window size (Default 5)
     - min_count: Ignore words with number of occurrences below this (Default 5)
     - loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
     - sample: Threshold for downsampling higher-frequency words (Default 0.001)
     - negative: Number of negative words to sample, for `ns` (Default 5)
     - iter: Number of epochs (Default 5)
     - sorted_vocab: Sort vocab by descending frequency (Default 1)
     - threads: Number of threads to use (Default 12)
     
In addition, FastText has **three additional parameters** - 
    - min_n: min length of char ngrams (Default 3)
    - max_n: max length of char ngrams (Default 6)
    - bucket: number of buckets used for hashing ngrams (Default 2000000)
Parameters `min_n` and `max_n` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If `max_n` is set to 0, or to be lesser than `min_n`, no character ngrams are used, and the model effectively reduces to Word2Vec.
  
To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the [Fowler-Noll-Vo hashing function](http://www.isthe.com/chongo/tech/comp/fnv) (FNV-1a variant) is employed.

**Note:** As in the case of Word2Vec, you can continue to train your model while using Gensim's native implementation of fastText. However, continuation of training with fastText models while using the wrapper is not supported.

## Saving/loading models

Models can be saved and loaded via the `load` and `save` methods.

In [21]:
# saving a model trained via Gensim's fastText implementation
model_gensim.save('saved_model_gensim')
loaded_model = FT_gensim.load('saved_model_gensim')
print(loaded_model)

FastText(vocab=1762, size=100, alpha=0.025)


The `save_word2vec_method` causes the vectors for ngrams to be lost. As a result, a model loaded in this way will behave as a regular word2vec model.  


## Word vector lookup
**Note:** Operations like word vector lookups and similarity queries can be performed in exactly the same manner for both the implementations of fastText so they have been demonstrated using only the fastText wrapper here.
**FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.**

In [22]:
print('night' in model_gensim.wv.vocab)
print('nights' in model_gensim.wv.vocab)
print(model_gensim['night'])
print(model_gensim['nights'])

True
False
[-0.57019579 -0.09188829  0.20156042 -0.3072716  -0.39754426 -0.0983983
 -0.08722843 -0.41355431 -0.36677885 -0.24853006 -0.52013594 -0.75637078
 -0.0158626   0.23842414  0.21981972  0.03080155 -0.40407965 -0.09736002
  0.27523348 -0.30056232 -0.18164943 -0.41765156 -0.0892433  -0.24988179
 -0.39415315  0.20866108 -0.09102312  0.07848851 -0.32144812  0.73408467
 -0.12648621 -0.53506029 -0.21370618 -0.0779781   0.53826493 -0.43128857
 -0.32431749 -0.05279015 -0.8310082   0.21771659  0.22414613  0.58643883
 -0.57156646 -0.01516446 -0.03859856 -0.20099074 -0.33015272  0.02311516
 -0.05932569 -0.13622418  0.00897878  0.36167803  0.0329816   0.37954178
  0.71049422 -0.13055792 -0.76480305  0.20057715  0.05422566 -0.05715587
  0.82215869 -0.13244535 -0.98220849 -0.22568038  0.16175243  0.01689859
 -0.53634089  0.14001484 -0.16060339 -0.52491415 -0.14224397 -0.03877361
 -0.37828463  0.15975705 -0.4798829   0.41982552 -0.3067295   0.61016631
  0.29826355  0.38651058 -0.39210784 -0.1

#### The word vector lookup operation only works if at least one of the component character ngrams is present in the training corpus. For example -

In [23]:
# Raises a KeyError since none of the character ngrams of the word `axe` are present in the training data
model_gensim['axe']

KeyError: 'all ngrams for word axe absent from model'

The `in` operation works slightly differently from the original word2vec.  
It tests whether a vector for the given word exists or not, not whether the word is present in the word vocabulary.  
To test whether a word is present in the training word vocabulary.

In [25]:
# Tests if word present in vocab
print("word" in model_gensim.wv.vocab)
# Tests if vector present for word
print("word" in model_gensim)

False
True


/home/saucecat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.


## Similarity operations

Similarity operations work the same way as word2vec.  
**Out-of-vocabulary words can also be used, provided they have at least one character ngram present in the training data.**

In [27]:
print("nights" in model_gensim.wv.vocab)
print("night" in model_gensim.wv.vocab)
model_gensim.wv.similarity("night", "nights")

False
True


/home/saucecat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0.99999390394357246

Syntactically similar words generally have high similarity in fastText models, since a large number of the component char-ngrams will be the same.  
As a result, fastText generally does better at syntactic tasks than Word2Vec. A detailed comparison is provided [here](Word2Vec_FastText_Comparison.ipynb).

### Other similarity operations

In [32]:
# The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only
model_gensim.wv.most_similar("nights")

[(u'night', 0.9999696016311646),
 (u'night.', 0.9999635219573975),
 (u'rights', 0.999962568283081),
 (u'office', 0.9999616146087646),
 (u'might', 0.9999598264694214),
 (u'flights', 0.9999597668647766),
 (u'territories', 0.9999595880508423),
 (u'warning', 0.9999595284461975),
 (u'contested', 0.9999594688415527),
 (u'offices', 0.9999593496322632)]

In [33]:
model_gensim.wv.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant'])

0.99996431181513856

In [34]:
model_gensim.wv.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [36]:
model_gensim.wv.most_similar(positive=['baghdad', 'england'], negative=['london'])

[(u'Zealand', 0.9995945692062378),
 (u'jail', 0.9995880722999573),
 (u'bomber', 0.9995837807655334),
 (u'Yasser', 0.9995830655097961),
 (u'bombers', 0.9995797872543335),
 (u'land', 0.9995788335800171),
 (u'shuttle', 0.9995787143707275),
 (u'New', 0.9995783567428589),
 (u'water', 0.9995779395103455),
 (u'major', 0.999577522277832)]

### word embedding evaluation

In [38]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [39]:
question_file_path = data_dir + 'questions-words.txt'
accuracy_results = model_gensim.accuracy(questions=question_file_path)

2018-01-09 17:20:08,273 : INFO : family: 0.0% (0/2)
2018-01-09 17:20:08,314 : INFO : gram3-comparative: 16.7% (2/12)
2018-01-09 17:20:08,344 : INFO : gram4-superlative: 25.0% (3/12)
2018-01-09 17:20:08,379 : INFO : gram5-present-participle: 30.0% (6/20)
2018-01-09 17:20:08,408 : INFO : gram6-nationality-adjective: 40.0% (8/20)
2018-01-09 17:20:08,440 : INFO : gram7-past-tense: 10.0% (2/20)
2018-01-09 17:20:08,465 : INFO : gram8-plural: 33.3% (4/12)
2018-01-09 17:20:08,476 : INFO : total: 25.5% (25/98)
